In [1]:
import os
import glob
import json
from tld import get_fld

In [2]:
# Constants
DATA_DIR = '../crawl_data'
SRC_DIR = '../crawler_src'

In [3]:
# Load the datasets
os.chdir(DATA_DIR)

# Get desktop json files
data_json_desktop = glob.glob('*_desktop.json')

# Get mobile json files
data_json_mobile = glob.glob('*_mobile.json')

In [4]:
def init_data_object():
    return  {
        # Per url data
        'tranco_ranks': [],
        'page_load_times': [],
        'num_requests': [],
        'distinct_third_parties': [],
        'num_distinct_tracker_domains': [],
        'num_distinct_tracker_entities': [],
        
        # Global data
        'failures': {
            'timeout_failures': 0,
            'TLS_failures': 0,
            'consent_failures': 0
        },
        'third_party_counts': {},
        'third_party_tracker_counts': {},
        'third_party_tracker_entities': {},
        'uber_cookie': {
            'request_hostname': '',
            'website': '',
            'num_cookies': 0,
            'first_party': False
        },
        'longest_lifespan_cookies': []
    }

In [5]:
def parse_stupid_blocklist_to_something_readable(file_path):
    url_list = {}
    
    with open(file_path) as blocklist_file:
        blocklist = json.load(blocklist_file)
        
        for cat, entities in blocklist['categories'].items():
            for entity_list in entities:
                for entity, url_objects in entity_list.items():
                    for url, aliases in url_objects.items():
                        all_urls = [url]
                        all_urls += aliases
                        
                        if entity not in url_list:
                            url_list[entity] = []
                            
                        for u in all_urls:
                            
                            try:
                                url_list[entity].append(get_fld(u, fix_protocol=True))
                            except:
                                pass
                        
                        url_list[entity] = list(set(url_list[entity]))
    
    return url_list

In [15]:
def create_stats_object(json_files):
    data_object = init_data_object()
    
    for json_file in json_files:
        with open(json_file, 'r') as data_file:
            data = json.load(data_file)
            
            # Append tranco rank
            data_object['tranco_ranks'].append(data['rank'])
            
            # Updata failure counts
            if data['failure_status']['timeout']:
                data_object['failures']['timeout_failures'] += 1
            if data['failure_status']['TLS'] != 'null':
                data_object['failures']['TLS_failures'] += 1
            if data['failure_status']['consent']:
                data_object['failures']['consent_failures'] += 1
                
            # Only proceed if there is no timeout
            if data['failure_status']['timeout']:
                data['num_requests'].append(None)
                data['distinct_third_parties'].append(None)
                data['num_distinct_tracker_domains'].append(None)
                data['num_distinct_tracker_entities'].append(None)
                continue
            
            # Append page load time
            data_object['page_load_times'].append(data['load_time'])
            
            # Append number of requests
            data_object['num_requests'].append(len(data['requests']))
            
            # Append distinct third parties
            distinct_third_parties = set([ get_fld(d['request_url'], fix_protocol=True)
                                           for d in data['requests']
                                         ])
            distinct_third_parties.remove(get_fld(data['website_domain'], fix_protocol=True))
            data_object['distinct_third_parties'].append(len(distinct_third_parties))
            
            # Append number of distinct tracker domains
            tracker_dict = parse_stupid_blocklist_to_something_readable(SRC_DIR + '/disconnectmeblocklist.json')
            distinct_tracker_domains = []
            for third_party_domain in distinct_third_parties:
                for _, domains in tracker_dict.items():
                    if third_party_domain in domains:
                        distinct_tracker_domains.append(third_party_domain)
            data_object['num_distinct_tracker_domains'].append(len(distinct_tracker_domains))
            
            # Append number of distinct tracker entities/companies
            distinct_tracker_entities = []
            with open(SRC_DIR + '/domain_map.json') as domain_map_json_file:
                domain_map_json = json.load(domain_map_json_file)
                
                for third_tracker_domain in distinct_tracker_domains:
                    if third_tracker_domain in domain_map_json.keys():
                        distinct_tracker_entities.append(domain_map_json[third_party_domain]['entityName'])
                    else:
                        print('DOMAIN NOT FOUND IN ENTITY LIST, IMPLEMENT THIS WHEN YOU SEE THIS! DOMAIN OF DOOM: {}'
                              .format(third_party_domain))
                
            distinct_tracker_entities = set(distinct_tracker_entities)
            data_object['num_distinct_tracker_entities'].append(len(distinct_tracker_entities))
            
            # Update third party reference counts
            for party in distinct_third_parties:
                if party in data_object['third_party_counts']:
                    data_object['third_party_counts'][party] += 1
                else:
                    data_object['third_party_counts'][party] = 1
            
            # Update third party tracker counts
            for tracker in distinct_tracker_domains:
                if tracker in data_object['third_party_tracker_counts']:
                    data_object['third_party_tracker_counts'][tracker] += 1
                else:
                    data_object['third_party_tracker_counts'][tracker] =1
            
            # Update third party tracker entities
            for entity in distinct_tracker_entities:
                if entity in data_object['third_party_tracker_entities']:
                    data_object['third_party_tracker_entities'][entity] += 1
                else:
                    data_object['third_party_tracker_entities'][entity] =1
                    
            # Update the uber cookie
            max_cookie_count = None
            request_url = None
            for request in data['requests']:
                if 'cookie' not in request['request_headers']:
                    continue
                    
                cookie_count = len(request['request_headers']['cookie'].split(';'))
                if max_cookie_count is None or cookie_count > max_cookie_count:
                    max_cookie_count = cookie_count
                    request_url = request['request_url']
            
            if max_cookie_count is not None and \
               max_cookie_count > data_object['uber_cookie']['num_cookies']:
                data_object['uber_cookie']['num_cookies'] = max_cookie_count
                data_object['uber_cookie']['request_url'] = get_fld(request_url, fix_protocol=True)
                data_object['uber_cookie']['website'] = data['website_domain']
                data_object['uber_cookie']['first_party'] = data_object['uber_cookie']['request_url'] == \
                                                            data_object['uber_cookie']['website']
                
            # Get longest lasting cookies
            # TODO: FIX THIS
            cookie_ids = []
            for request in data['requests']:
                if 'cookie' not in request['request_headers']:
                    continue
                    
                cookies = request['request_headers']['cookie'].split(';')
                for cookie in cookies:
                    cookie_ids.append(cookie.split('=')[0])
                
            # TODO HTTP redirect pairs for tracker domains
            # TODO Fingerprints
            
    return data_object
                
# TODO: Remove
create_stats_object(data_json_desktop)

{'failures': {'timeout_failures': 0, 'TLS_failures': 1, 'consent_failures': 0},
 'tranco_ranks': [None],
 'page_load_times': [6.0],
 'num_requests': [50],
 'distinct_third_parties': [1],
 'num_distinct_tracker_domains': [1],
 'num_distinct_tracker_entities': [1],
 'third_party_counts': {'google.com': 1},
 'third_party_tracker_counts': {'google.com': 1},
 'third_party_tracker_entities': {'Google LLC': 1},
 'uber_cookie': {'request_hostname': '',
  'website': 'amiunique.org',
  'num_cookies': 3,
  'first_party': True,
  'request_url': 'amiunique.org'},
 'longest_lifespan_cookies': []}

In [6]:
# Desktop analysis